# Postage Stamps

This notebook creates postage stamps for subtracted images generates by the Diference Image Analysis DIA pipeline. For instructions on running the pipeline, see this project's ReadMe.


## Setup

You will need to specify the directory of the differenced image you wish to process. This directory should contain (or contain subdirectories with) both the differnece image itself, along with the object cataloge for that image (which is autoimatically generated byt the DIA pipeline).

In [ ]:
# Input DIA data
diff_im_dir = '/global/u1/d/djp81/test_imdiff/'

# Output directory for postave stamps
postage_output_dir = './stamps'


In [ ]:
import os

import numpy as np
from astropy.visualization import ZScaleInterval
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.table import Table, vstack
from astropy.wcs import WCS
from pathlib import Path

import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
from desc_dc2_dm_data import REPOS


## Getting Target List

Subtraction results from the DIA pipeline are saved as pairs of *diffexp* and *diaSrc* files. The *diffexp* files contain the subracted image data while the *diaSrc* files act as a catlog of objects found in the subtracted image. We collect the paths of any DIA output files in the directory `diff_im_dir` specified above and create a combined table of detected objects.

In [ ]:
def get_diff_file_paths(dia_dir):
    """Return lists of all diffexp and diaSrc file paths under ``dia_dir``
    
    Args:
        dia_dir (str): Path of DIA pipeline outputs
        
    Returns:
        A list of all diffexp paths as Path objects
        A list of all diaSrc paths as Path objects
    """

    diff_files, dia_files = [], []
    for filename in Path(dia_dir).glob('**/diffexp*.fits'):
        diff_files.append(filename)
        dia_files.append(Path(str(filename).replace('diffexp', 'diaSrc')))
        
    return diff_files, dia_files


diffexp_paths, diasrc_paths = get_diff_file_paths(diff_im_dir)

print('diffexp paths:')
print(diffexp_paths)

print('\ndiaSrc paths:')
print(diasrc_paths, '\n')

In [ ]:
def read_diasrc_path(path):
    """Read a diaSrc file and return a table of detected sources
    
    Args:
        path (str): Path of the file to read
        
    Returns:
        An astropy table of source data, including coordinates
    """

    diaSrc_hdu_list = fits.open(path)
    diaSrc_data = diaSrc_hdu_list[1].data
    return Table(np.array(diaSrc_data))
    

def read_diasrc_path_list(diasrc_list):
    """Read multiple diaSrc files and return a table of all detected sources
    
    Args:
        diasrc_list (list[str]): List of diaSrc paths to read
        
    Returns:
        An astropy table of all detected sources
    """
        
    diasrc_list = list(diasrc_list)  # Copy to avoid mutation
    data_table = read_diasrc_path(diasrc_list.pop())
    for path in diasrc_list:
        data_table = vstack(data_table, read_diasrc_path(path))
        
    return data_table

# First ten sources as an example
read_diasrc_path_list(diasrc_paths)[:10]


## Creating Postage Stamps

We port some logic for postage stamp creation from the [DC2 postage stamp tutorial](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/dm_butler_postage_stamps.ipynb) by Michael Wood-Vasey. 

In [ ]:
def cutout_coadd_ra_dec(butler, ra, dec, filter='r', datasetType='deepCoadd', **kwargs):
    """Produce a cutout from coadd from the given butler at the given RA, Dec in decimal degrees.
    
    Args:
        butler (Butler): Servant providing access to a data repository
        ra      (float): Right ascension of the center of the cutout, degrees
        dec     (float): Declination of the center of the cutout, degrees
        filter (string): Filter of the image to load
        
    Returns:
        MaskedImage
    """

    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)
    return cutout_coadd_spherepoint(butler, radec, filter=filter, datasetType=datasetType)
    

def cutout_coadd_spherepoint(butler, radec, filter='r', datasetType='deepCoadd',
                             skymap=None, SideLength=51, **kwargs):
    """Produce a cutout from a coadd at the given afw SpherePoint radec position.
    
    Args:
        butler      (Butler): Servant providing access to a data repository
        radec  (SpherePoint): Coordinates of the center of the cutout.
        filter      (string): Filter of the image to load
        datasetType (string): Which type of coadd to load. Doesn't support 'calexp' (Default: 'deepCoadd')
        skymap      (SkyMap): Pass in to avoid the Butler read. Useful if you have lots of them. (Default: None)
        SideLength   (float): Side of the cutout region in pixels. (Default: None)
    
    Returns:
        MaskedImage
    """

    cutoutSize = afwGeom.ExtentI(SideLength, SideLength)

    if skymap is None:
        skymap = butler.get("%s_skyMap" % datasetType)
    
    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = afwGeom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = afwGeom.BoxI(xy - cutoutSize//2, cutoutSize)

    coaddId = {'tract': tractInfo.getId(), 'patch': "%d,%d" % patchInfo.getIndex(), 'filter': filter}
    
    cutout_image = butler.get(datasetType+'_sub', bbox=bbox, immediate=True, dataId=coaddId)
    
    return cutout_image

def make_cutout_image(butler, ra, dec, filter='r', vmin=None, vmax=None, label=None,
                      show=True, saveplot=False, savefits=False,
                      datasetType='deepCoadd'):
    """Generate and optionally display and save a postage stamp for a given RA, Dec.
    
    Args:
        butler (Butler): Servant providing access to a data repository
        ra      (float): Right ascension of the center of the cutout, degrees
        dec     (float): Declination of the center of the cutout, degrees
        filter (string): Filter of the image to load
   
    Returns:
        MaskedImage
    """

    cutout_image = cutout_coadd_ra_dec(butler, ra, dec, filter=filter, datasetType=datasetType)
    if savefits:
        if isinstance(savefits, str):
            filename = savefits
        else:
            filename = 'postage-stamp.fits'
        cutout_image.writeFits(filename)
    
    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)
    xy = cutout_image.getWcs().skyToPixel(radec)
    
    if vmin is None or vmax is None:
        vmin, vmax = zscale.get_limits(cutout_image.image.array)

    plt.imshow(cutout_image.image.array, vmin=vmin, vmax=vmax, cmap='binary_r', origin='lower')
    plt.colorbar()
    plt.scatter(xy.getX() - cutout_image.getX0(), xy.getY() - cutout_image.getY0(),
                color='none', edgecolor='red', marker='o', s=200)
    
    if label is not None:
        plt.title(label)
        
    if saveplot:
        if isinstance(saveplot, str):
            filename = saveplot

        else:
            filename = 'postage-stamp.png'

        plt.savefig(filename)

    if show:
        plt.show()

    return cutout_image


Next we create postage stamps for each detected source.

In [ ]:
def create_postage_stamps(dia_dir, out_dir, butler=None):
    """Create postage stamps for all dia source objects

    Args:
        dia_dir   (str): Path of DIA pipeline outputs
        out_dir   (str): Output directory of postage stamp fits files
        butler (Butler): Optionally use an existing butler instead of
            initializing a new one
    """
    
    if butler is None:
        print('Instantializing butler...')
        butler = dafPersist.Butler(dia_dir)
    
    print('Retrieving DIA sources')
    diffexp_paths, diasrc_paths = get_diff_file_paths(dia_dir)
    source_table = read_diasrc_path_list(diasrc_paths)
    
    print('Creating Postage stamps...')
    for source in source_table:
        make_cutout_image(butler, source['coord_ra'], source['coord_dec'], datasetType='deepDiff', show=True)
        break
 

In [ ]:
butler = dafPersist.Butler(diff_im_dir)
create_postage_stamps(diff_im_dir, postage_output_dir, butler=butler)
